In [90]:
# Load libraries
from importlib import reload 
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import scipy.io as si
import scipy.sparse as ss
from gcMapExplorer import lib as gmlib
from tqdm import tqdm
import pickle
import process_hic_interX as ph
reload(ph)

<module 'process_hic_interX' from '/home/louiscam/projects/gpcr/code/tcell_processing/process_hic_interX.py'>

In [83]:
# Directories
unprocessed_hic_dir = '/home/louiscam/projects/gpcr/data/hic/CD3_Tcells/MAPQ30_KRnormalized/'
processed_hic_dir = '/home/louiscam/projects/gpcr/data/hic/CD3_Tcells/MAPQ30_fullprocessed/'
genome_dir = '/home/louiscam/projects/gpcr/data/genome_data/'
hic_plots_dir = '/home/louiscam/projects/gpcr/data/hic/CD3_Tcells/MAPQ30_hicplots/'

In [68]:
# Characteristics
resol = 250000
chr_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
filter_size = 2000000

### Load complementary data

In [69]:
# Get chromosome size
df_sizes = ph.get_chrom_sizes(genome_dir,resol)
# Get repeats data
df_repeats = ph.load_repeats_data(genome_dir)
# Identify repeat locations to filter out
dic_repeats_tofilter = ph.find_repeat_locations(df_repeats, chr_list, df_sizes, resol)
# Load centromere information
df_centrom = ph.get_centromere_locations(genome_dir)

### Load data

In [93]:
# Construct data directory
chr1 = 1
chr2 = 2

# Load raw data
KRnorm_hic_data = ph.get_norm_hic_sparse(unprocessed_hic_dir, resol, chr1, chr2)

# Get chromosome sizes (in number of loci)
chr1_size = int(df_sizes[df_sizes['chr']==str(chr1)]['size_loci'])
chr2_size = int(df_sizes[df_sizes['chr']==str(chr2)]['size_loci'])

# Construct normalized dense HiC dataframe
df = ph.get_dense_hic_dataframe(KRnorm_hic_data, chr1_size, chr2_size, resol)

# Filter out centromeric and pericentromeric regions
df = ph.filter_centromeres(df, chr1, 'row', df_centrom, filter_size, resol)
df = ph.filter_centromeres(df, chr2, 'col', df_centrom, filter_size, resol)

# Filter repeats for chr1 and chr2
df = ph.filter_repeats(df, chr1, dic_repeats_tofilter, 'row')
df = ph.filter_repeats(df, chr2, dic_repeats_tofilter, 'col')

# Log-transform dataframe
df_transformed = ph.log_transform(df)

# Filter out outliers
df_transformed = ph.filter_outliers(df_transformed)

# Plot and save to pickle HiC data after filtering out centromeres, repeats and outliers
plotname = 'hic_'+'chr'+str(chr1)+'_'+'chr'+str(chr2)+'_norm1_filter3'+'.eps'
ph.plot_dense_hic_dataframe(df_transformed, chr1, chr2, plotname, hic_plots_dir)
picklename = processed_hic_dir+'hic_'+'chr'+str(chr1)+'_'+'chr'+str(chr2)+'_norm1_filter3'+'.pkl'
pickle.dump(df_transformed, open(picklename, 'wb'))